In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline

from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTEENN

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler



test = pd.read_csv(r'C:\Users\de969\OneDrive\Escritorio\proyecto, machine learnig\_marketing\data\test.csv',sep='\t')


In [11]:
median_income = test['Income'].median()
test['Income'].fillna(median_income, inplace=True)

In [19]:
import pickle
from pathlib import Path

def read_pickle(file_path: str):
    """
    Read data from a Pickle file.

    Args:
        file_path (str): Path of the Pickle file.

    Returns:
        object: Data loaded from the Pickle file, or None if there is an error.
    """
    try:
        with open(file_path, "rb") as f:
            data = pickle.load(f)
        print("Pickle file read: OK")
        return data
    except (FileNotFoundError, IOError, pickle.PickleError) as err:
        print(f"Failed to read Pickle file {file_path}: {err}")
        return None

# Leer el modelo desde el archivo Pickle

import joblib

# Cargar el modelo desde el archivo .pkl
loaded_model = joblib.load(r'C:\Users\de969\OneDrive\Escritorio\proyecto, machine learnig\_marketing\models\mejor_modelo.pkl')


# modelo = read_pickle("_marketing\\models\\mejor_modelo.pkl")

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# import pandas as pd

# test = pd.read_csv(r'C:\Users\de969\OneDrive\Escritorio\proyecto, machine learnig\_marketing\data\test.csv',sep='\t')
# median_income = test['Income'].median()
# test['Income'].fillna(median_income, inplace=True)

# features = ['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines',
#             'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
#             'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
#             'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
#             'Z_CostContact', 'Z_Revenue'] 
# target = 'Response' 

# X = test[features]
# y = test[target]
# # predicciones
# predictions = modelo.predict(X)



# from sklearn.metrics import roc_auc_score
# accuracy = accuracy_score(y, predictions)
# precision = precision_score(y, predictions)
# recall = recall_score(y, predictions)
# f1 = f1_score(y, predictions)
# roc_auc = roc_auc_score(y, predictions)

In [28]:


# Seleccionar las características y el objetivo
features = ['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines',
            'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
            'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
            'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
            'Z_CostContact', 'Z_Revenue']  # Variables predictoras
target = 'Response'  # Variable objetivo

X_test = test[features]
y_test = test[target]

predictions = loaded_model.predict(X_test)

from sklearn.metrics import roc_auc_score
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, predictions)
print(accuracy)
print(precision)
print(recall)
print(f1)
print(roc_auc)


0.796875
0.35789473684210527
0.53125
0.4276729559748428
0.6861979166666667


In [16]:


# # Crear el pipeline con escalado de características, técnica de resampling y modelo de clasificación
# pipeline_smote_tomek = Pipeline([
#     ('scaler', StandardScaler()),
#     ('resampler', SMOTE(sampling_strategy='auto')),
#     ('undersampler', TomekLinks(sampling_strategy='majority')),
#     ('classifier', RandomForestClassifier())
# ])

# # Definir los parámetros del grid search
# param_grid_smote_tomek = {
#     'classifier__n_estimators': [50, 100, 200]
# }

# # Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
# grid_search_smote_tomek = GridSearchCV(pipeline_smote_tomek, param_grid_smote_tomek, cv=5, scoring='accuracy')
# grid_search_smote_tomek.fit(X_train, y_train)

# # Obtener los resultados de la búsqueda de hiperparámetros
# best_model_smote_tomek = grid_search_smote_tomek.best_estimator_
# best_params_smote_tomek = grid_search_smote_tomek.best_params_
# best_score_smote_tomek = grid_search_smote_tomek.best_score_

# # Imprimir los resultados
# print("Resultados de la búsqueda de hiperparámetros para SMOTE + TomekLinks:")
# print("Mejor modelo:", best_model_smote_tomek)
# print("Mejores parámetros:", best_params_smote_tomek)
# print("Mejor puntuación:", best_score_smote_tomek)